In [ ]:
!pip install streamlit pyjwt bcrypt pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.0 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.0
    Uninstalling cachetools-7.0.0:
      Successfully uninstalled cachetools-7.0.0


In [ ]:
%%writefile database.py
import sqlite3

def create_db():
    conn = sqlite3.connect("policy_nav.db")
    c = conn.cursor()
    c.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT,
        email TEXT UNIQUE,
        password BLOB,
        security_question TEXT,
        security_answer TEXT
    )
    """)
    conn.commit()
    conn.close()


Writing database.py


In [ ]:
%%writefile auth.py
import jwt
import bcrypt
from datetime import datetime, timedelta

SECRET_KEY = "policynav_secret"

def hash_password(password):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt())

def check_password(password, hashed):
    return bcrypt.checkpw(password.encode(), hashed)

def generate_token(email):
    payload = {
        "email": email,
        "exp": datetime.utcnow() + timedelta(hours=1)
    }
    return jwt.encode(payload, SECRET_KEY, algorithm="HS256")

Writing auth.py


In [ ]:
%%writefile app.py
import streamlit as st
import sqlite3
from database import create_db
from auth import *
import re

def local_css():
    st.markdown("""
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@700;800&family=Poppins:wght@400;600&display=swap');

    header[data-testid="stHeader"] {
    display: none;
    }

    .stApp {
        background-color: #000000;
        padding-top: 0px;
    }

    h1 {
    font-family: 'Montserrat', sans-serif !important;
    text-align: center !important;
    font-weight: 800 !important;
    text-transform: uppercase;
    letter-spacing: 3px;
    margin-top: -50px !important;
    margin-bottom: 20px !important;

    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    background-clip: text;

    text-shadow: none;
    }


    [data-testid="stSidebar"] {
        display: none;
    }

    .stMainBlockContainer {
        background-color: rgba(255, 255, 255, 0.95);
        padding: 3rem 2rem;
        border-radius: 20px;
        box-shadow: 0 20px 40px rgba(0,0,0,0.4);
        max-width: 480px;
        margin: auto;
    }

    div.stButton > button:first-child {
        background-color: transparent;
        color: #764ba2;
        border: 2px solid #764ba2;
        font-weight: 600;
        transition: 0.3s;
    }

    div.stButton > button:first-child:hover {
        background-color: #764ba2;
        color: white;
    }

    .stButton>button[kind="primary"] {
        width: 100%;
        background-color: #764ba2;
        color: white;
        border-radius: 8px;
        height: 3.2em;
        font-family: 'Poppins', sans-serif;
        border: none;
    }

    .forgot-btn-container {
        text-align: right;
        margin-top: -10px;
    }

    button[key*="forgot"] {
        background: none !important;
        border: none !important;
        color: #764ba2 !important;
        text-decoration: underline !important;
        font-size: 14px !important;
        font-weight: 400 !important;
        padding: 0 !important;
        box-shadow: none !important;
    }

    h3 {
        font-family: 'Montserrat', sans-serif !important;
        font-weight: 700 !important;
        text-align: center;

        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
    }


    label {
        font-family: 'Poppins', sans-serif;
        font-weight: 600 !important;
        color: #444 !important;
    }
    </style>
    """, unsafe_allow_html=True)

local_css()

def is_valid_email(email):
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email)

create_db()
conn = sqlite3.connect("policy_nav.db", check_same_thread=False)
c = conn.cursor()

st.title("policyNav")

if "page" not in st.session_state:
    st.session_state["page"] = "Login"

col1, col2 = st.columns(2)
with col1:
    if st.button("Login", use_container_width=True):
        st.session_state["page"] = "Login"
with col2:
    if st.button("Signup", use_container_width=True):
        st.session_state["page"] = "Signup"

st.markdown("<br>", unsafe_allow_html=True)

# LOGIN PAGE
if st.session_state["page"] == "Login":
    email = st.text_input("Email ID", placeholder="example@mail.com").strip().lower()
    password = st.text_input("Password", type="password")

    if st.button("Log In", type="primary", use_container_width=True):
        c.execute("SELECT username, password FROM users WHERE email=?", (email,))
        user = c.fetchone()
        if user and check_password(password, user[1]):
            st.session_state["user"] = user[0]
            st.success(f"Welcome {user[0]}!")
            st.balloons()
        else:
            st.error("Invalid credentials")

    st.markdown("<br>", unsafe_allow_html=True)

    if st.button("Forgot Password?", key="forgot_login_btn", use_container_width=True):
        st.session_state["page"] = "Forgot Password"
        st.rerun()



# SIGNUP PAGE
elif st.session_state["page"] == "Signup":
    username = st.text_input("Username")
    email = st.text_input("Email ID").strip().lower()
    password = st.text_input("Password", type="password")
    confirm = st.text_input("Confirm Password", type="password")

    question = st.selectbox("Security Question", [
        "What is your pet name?",
        "What is your favorite subject?",
        "What is your favorite holiday destination?"
    ])
    answer = st.text_input("Security Answer").strip().lower()

    left, center, right = st.columns([1, 2, 1])

    with center:
        if st.button("Register", type="primary", use_container_width=True):
            if not all([username, email, password, confirm, answer]):
                st.error("All fields are mandatory")
            elif not is_valid_email(email):
                st.error("Invalid email address")
            elif not password.isalnum():
              st.error("Password must be Alpha-Numeric")
            elif len(password)<8:
              st.error("Password Must contain atleast 8 characters")
            elif password != confirm:
                st.error("Passwords do not match")
            else:
                try:
                    c.execute(
                        "INSERT INTO users VALUES (NULL,?,?,?,?,?)",
                        (username, email, hash_password(password), question, answer)
                    )
                    conn.commit()
                    st.success("Signup successful! Please login.")
                    st.session_state["page"] = "Login"
                except sqlite3.IntegrityError:
                    st.error("Email already exists")

# FORGOT PASSWORD PAGE
elif st.session_state["page"] == "Forgot Password":
    st.subheader("Reset Password")
    email_reset = st.text_input("Enter your registered Email").strip().lower()

    if st.button("Verify Email", type="primary"):
        if email_reset.strip() == "":
            st.error("Email field cannot be empty")

        else:
            c.execute(
                "SELECT security_question, security_answer FROM users WHERE email=?",
                (email_reset,)
            )
            data = c.fetchone()

            if data is None:
                st.error("No email found")

            else:
                st.session_state["reset_email"] = email_reset
                st.session_state["reset_question"] = data[0]
                st.session_state["reset_answer"] = data[1]


    if "reset_question" in st.session_state:
        st.info(f"Question: {st.session_state['reset_question']}")
        ans = st.text_input("Your Answer").strip().lower()
        new_pass = st.text_input("New Password", type="password")
        confirm_pass = st.text_input("Confirm Password", type="password")

        if st.button("Update Password", type="primary"):

          if ans != st.session_state["reset_answer"]:
              st.error("Incorrect security answer")

          elif new_pass.strip() == "" or confirm_pass.strip() == "":
              st.error("Password fields cannot be empty")

          elif len(new_pass) < 8:
              st.error("Password must be at least 8 characters long")

          elif not new_pass.isalnum():
              st.error("Password must contain only letters and numbers")

          elif new_pass != confirm_pass:
              st.error("New Password and Confirm Password do not match")

          else:

            c.execute(
                "SELECT password FROM users WHERE email=?",
                (st.session_state["reset_email"],)
            )
            existing_hashed_password = c.fetchone()[0]

            if check_password(new_pass, existing_hashed_password):
                st.error("Current password cannot be used again")

            else:
                c.execute(
                    "UPDATE users SET password=? WHERE email=?",
                    (hash_password(new_pass), st.session_state["reset_email"])
                )
                conn.commit()

                st.success("Password updated successfully! Redirecting to login...")
                st.session_state["page"] = "Login"

                del st.session_state["reset_question"]
                del st.session_state["reset_answer"]
                del st.session_state["reset_email"]



    if st.button("Back to Login"):
        st.session_state["page"] = "Login"
        st.rerun()

Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("YOUR_NGROK_AUTHTOKEN")

In [ ]:
public_url = ngrok.connect(8501)
print("OPEN THIS URL:", public_url)

OPEN THIS URL: NgrokTunnel: "https://shannan-unpropitiatory-ming.ngrok-free.dev" -> "http://localhost:8501"
